In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Resnet152


In [ ]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np
from keras.applications.resnet_v2 import ResNet152V2

# ResNet152 모델 로드 (최상위 층 제외)
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('Resnet152_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

234545216/234545216 [==============================] - 7s 0us/step
Found 1502 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 672s 6s/step - loss: 5.5610 - accuracy: 0.5180 - val_loss: 5.4683 - val_accuracy: 0.3599 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 17s 177ms/step - loss: 5.1078 - accuracy: 0.7250 - val_loss: 5.2761 - val_accuracy: 0.6013 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 17s 177ms/step - loss: 4.5655 - accuracy: 0.9201 - val_loss: 5.2041 - val_accuracy: 0.6422 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 17s 177ms/step - loss: 4.2267 - accuracy: 0.9621 - val_loss: 5.1478 - val_accuracy: 0.6638 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 17s 177ms/step - loss: 3.9414 - accuracy: 0.9614 - val_loss: 4.3736 - val_accuracy: 0.7457 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Average Validation Accuracy: 0.7226293101906777


## Mobilenet_V3

In [ ]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np
from keras.applications import MobileNetV3Large

# MobileNetV3Large 모델 로드 (최상위 층 제외)
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('mobilenet_v3_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

12683000/12683000 [==============================] - 1s 0us/step
Found 1502 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 33s 77ms/step - loss: 5.0946 - accuracy: 0.5120 - val_loss: 4.7790 - val_accuracy: 0.5000 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 5s 58ms/step - loss: 4.8003 - accuracy: 0.6418 - val_loss: 4.7491 - val_accuracy: 0.5000 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 5s 57ms/step - loss: 4.4643 - accuracy: 0.7770 - val_loss: 4.6929 - val_accuracy: 0.5000 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 6s 58ms/step - loss: 4.1687 - accuracy: 0.8748 - val_loss: 4.6141 - val_accuracy: 0.5000 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 6s 59ms/step - loss: 3.9128 - accuracy: 0.9434 - val_loss: 4.5401 - val_accuracy: 0.5000 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 5s 57ms/s

## Inception_V3

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# InceptionV3 모델 로드 (최상위 층 제외)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))  # InceptionV3는 299x299 입력을 사용

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # kernel_regularizer는 필요에 따라 추가
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(299, 299),  # InceptionV3에 맞게 이미지 크기 조정
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(299, 299),  # InceptionV3에 맞게 이미지 크기 조정
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('InceptionV3_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

87910968/87910968 [==============================] - 3s 0us/step
Found 1502 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 43s 147ms/step - loss: 1.1115 - accuracy: 0.4561 - val_loss: 0.7023 - val_accuracy: 0.5194 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 10s 106ms/step - loss: 0.5949 - accuracy: 0.7250 - val_loss: 0.6722 - val_accuracy: 0.5970 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 10s 106ms/step - loss: 0.2351 - accuracy: 0.9041 - val_loss: 0.5681 - val_accuracy: 0.6746 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 10s 106ms/step - loss: 0.1095 - accuracy: 0.9587 - val_loss: 0.6693 - val_accuracy: 0.7004 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 10s 106ms/step - loss: 0.0824 - accuracy: 0.9734 - val_loss: 0.3340 - val_accuracy: 0.8448 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 

## DenseNet201

In [ ]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# DenseNet201 모델 로드 (최상위 층 제외)
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('DenseNet201_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

Found 1502 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 114s 210ms/step - loss: 5.5980 - accuracy: 0.4820 - val_loss: 5.2205 - val_accuracy: 0.5259 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 14s 152ms/step - loss: 4.9139 - accuracy: 0.7523 - val_loss: 5.3752 - val_accuracy: 0.4957 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 14s 152ms/step - loss: 4.1909 - accuracy: 0.9254 - val_loss: 5.6805 - val_accuracy: 0.5022 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 14s 152ms/step - loss: 3.5883 - accuracy: 0.9727 - val_loss: 5.9611 - val_accuracy: 0.5043 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 14s 152ms/step - loss: 3.0190 - accuracy: 0.9740 - val_loss: 4.6141 - val_accuracy: 0.5582 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 14s 151ms/step - loss: 2.5157 - accuracy: 0.9814 - val_loss: 3.3

## EffcientB7

In [ ]:
from keras.applications.efficientnet import EfficientNetB7
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# EfficientNetB0 모델 로드 (최상위 층 제외)
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('EfficientNetB7_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

258076736/258076736 [==============================] - 9s 0us/step
Found 1502 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 846s 8s/step - loss: 1.0791 - accuracy: 0.4794 - val_loss: 0.7389 - val_accuracy: 0.5000 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 30s 321ms/step - loss: 0.6428 - accuracy: 0.6924 - val_loss: 0.7387 - val_accuracy: 0.5000 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 30s 320ms/step - loss: 0.3076 - accuracy: 0.8569 - val_loss: 0.8353 - val_accuracy: 0.2909 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 30s 320ms/step - loss: 0.1499 - accuracy: 0.9407 - val_loss: 0.6084 - val_accuracy: 0.7134 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 30s 320ms/step - loss: 0.1030 - accuracy: 0.9647 - val_loss: 1.0524 - val_accuracy: 0.4720 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Average Validation Accuracy: 0.729849137365818
